# Using EDSL to sense check data
This notebook provides example code for sense checking survey data using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and market research with AI agents and large language models. 

## Contents
Using a set of responses to a survey about online marketplaces as an example, we demonstrate EDSL methods for: 

1. Evaluating survey questions (e.g., for clarity and improvements)
2. Analyzing each respondent's set of answers (e.g., to summarize or identify sentiment, themes, etc.)
3. Reviewing each answer individually (e.g., to evaluate its relevance or usefulness)

## Coop
We also show how to post EDSL questions, surveys, results and notebooks (like this one) to [Coop: an integrated platform for creating and sharing LLM-based research](https://www.expectedparrot.com/explore). 

## How EDSL works
EDSL is a flexible library that can be used to perform a broad variety of research tasks. A typical workflow consists of the following steps:

* Construct questions  
* Add data to the questions (e.g., for data labeling tasks)
* Use an AI agent to answer the questions
* Select a language model to generate the answers
* Analyze results in a formatted dataset

## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library.
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) OR store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) provides examples of EDSL basic components. 

## Example data
Our example data is a CSV consisting of several questions and a few rows of responses.
Here we store it at the Coop and then re-import it:

In [1]:
from edsl import FileStore

In [2]:
fs = FileStore("marketplace_survey_results.csv")
info = fs.push()
info

{'description': 'File: marketplace_survey_results.csv',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/83edbe9f-ff45-4d96-b35d-e388e07f0c4e',
 'uuid': '83edbe9f-ff45-4d96-b35d-e388e07f0c4e',
 'version': '0.1.45.dev1',
 'visibility': 'unlisted'}

In [3]:
csv_file = FileStore.pull(info["uuid"])

## Creating questions about the data
There are many questions we might want to ask about the data, such as:

* Does this survey question have any logical or syntactical problems? {{ *question* }}
* What is the overall sentiment of this respondent's answers? {{ *responses* }}
* Is this answer responsive to the question that was asked? {{ *question* }} {{ *answer* }}

## Question types
EDSL comes with many common question types that we can select from based on the form of the response that we want to get back from the model: multiple choice, checkbox, linear scale, free text, etc. [Learn more about EDSL question types](https://docs.expectedparrot.com/en/latest/questions.html).

Here we construct `Question` objects for the questions that we want to ask about the data, using `{{ placeholders }}` for the information that we will add to the questions in the steps that follow:

In [4]:
from edsl import QuestionFreeText, QuestionMultipleChoice, QuestionYesNo

In [5]:
q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = "Describe any logical or syntactical problems in the following survey question: {{ question }}"
)

In [6]:
q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the overall sentiment of this respondent's survey answers? {{ responses }}",
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

In [7]:
q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = "Is this answer responsive to the question that was asked? Question: {{ question }} Answer: {{ answer }}"
)

## Adding survey data to the questions
Next we'll add our data to our questions. This can be done efficiently by creating a `ScenarioList` representing the data. The individual `Scenario` objects in the list can be constructed in a variety of ways depending on the information that we want to include in a particular question.

We start by calling the `from_csv()` method to create a `ScenarioList` for the data in its original form. We can see that this generates a `Scenario` dictionary for each respondent's set of answers with key/value pairs for the individual questions and answers: 

In [8]:
from edsl import ScenarioList

In [9]:
sl = ScenarioList.from_csv(csv_file.to_tempfile()) # replace with CSV file name if importing a local file
sl

,Respondent ID,What do you like most about using our online marketplace?,What is one feature you would like to see added to improve your shopping experience?,Can you describe a recent experience where you were dissatisfied with our service?,How do you feel about the current product search and filtering options?,Is there anything else you would like to share about your experience with us?
0,101,The wide variety of products and the ease of use.,It would be great to have a personalized recommendation system based on my browsing history.,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.",The search and filtering options are intuitive and work well for me.,"No, keep up the great work!"
1,102,I enjoy the simplicity of the interface.,A feature that helps compare similar products side by side would be useful.,No complaints here.,I find the product search to be pretty effective.,I think the sky is a beautiful shade of purple today.
2,103,The platform is user-friendly and offers a vast selection of products.,Would love to see an option to save and compare different products.,"My delivery was late by a few days, which was frustrating.",It’s okay.,No.


## Evaluating the questions
For our first question we want to create a `Scenario` for each survey question:

In [10]:
from edsl import QuestionFreeText, Survey

q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = "Describe any logical or syntactical problems in the following survey question: {{ question }}"
)

q_improved = QuestionFreeText(
    question_name = "improved",
    question_text = "Please draft an improved version of the survey question. Return only the revised question text."
)

survey = Survey([q_logic, q_improved]).add_targeted_memory(q_improved, q_logic)

The survey questions are the `parameters` of the `ScenarioList` created above:

In [11]:
questions = list(sl.parameters)
questions

['Can you describe a recent experience where you were dissatisfied with our service?',
 'Respondent ID',
 'What do you like most about using our online marketplace?',
 'What is one feature you would like to see added to improve your shopping experience?',
 'How do you feel about the current product search and filtering options?',
 'Is there anything else you would like to share about your experience with us?']

We can pass them to the `from_list()` method to create a new `ScenarioList`, specifying that the key for each `Scenario` will be `question` in order to match the parameter of our logic question:

In [12]:
sl_questions = ScenarioList.from_list("question", questions)
sl_questions

,question
0,Can you describe a recent experience where you were dissatisfied with our service?
1,Respondent ID
2,What do you like most about using our online marketplace?
3,What is one feature you would like to see added to improve your shopping experience?
4,How do you feel about the current product search and filtering options?
5,Is there anything else you would like to share about your experience with us?


We select a model to use, and then add the scenarios to the survey when we run it:

In [13]:
from edsl import Model

m = Model("gemini-1.5-flash")

In [14]:
results = survey.by(sl_questions).by(m).run()

Job UUID,cfcdacda-cb1b-4fc0-b030-6cdd0eefa0a0
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/cfcdacda-cb1b-4fc0-b030-6cdd0eefa0a0
Exceptions Report URL,None
Results UUID,80462061-1956-4414-ba3b-f9ca52104ce3
Results URL,https://www.expectedparrot.com/content/80462061-1956-4414-ba3b-f9ca52104ce3


This generates a dataset of `Results` that we can access with built-in methods for analysis:

In [15]:
results.select("question", "logic", "improved")

,scenario.question,answer.logic,answer.improved
0,Can you describe a recent experience where you were dissatisfied with our service?,"The question ""Can you describe a recent experience where you were dissatisfied with our service?"" has a few potential problems: * **Leading Question:** It presupposes dissatisfaction exists. Someone who had a positive experience might not answer honestly, or might struggle to recall a negative one even if a minor issue occurred. A neutral question would be better. * **Ambiguity of ""recent"":** ""Recent"" is subjective. What constitutes ""recent"" for one person (e.g., last week) might be distant for another (e.g., last month). Defining a timeframe (e.g., ""within the last month,"" ""since your last visit"") would improve clarity and comparability. * **Open-ended nature (potentially problematic):** While open-ended questions allow for rich responses, they can be difficult to analyze quantitatively. The responses might be too varied to easily categorize and summarize. Consider including a follow-up question with specific options (e.g., rating scales for different aspects of service). * **Assumes prior service interaction:** The question assumes the respondent has actually used the service. It might be beneficial to add a screening question first to determine if they have indeed interacted with the service.","In the past month, have you had any experiences with our service? If so, please rate your overall satisfaction."
1,Respondent ID,"""Respondent ID"" is not a survey question; it's a field for identifying the respondent's data. It's not meant to elicit an answer from the respondent, but rather to serve as a unique label for their responses. Therefore, the problem isn't logical or syntactical within the context of a question, but rather its inappropriate placement as a *question*. It should be treated as a data field, not a question in the survey instrument.",How satisfied are you with our product?
2,What do you like most about using our online marketplace?,"The main problem with the question ""What do you like most about using our online marketplace?"" is that it's **open-ended and lacks structure**. This leads to several potential issues: * **Difficult to analyze:** Responses will be highly varied and unstructured, making quantitative analysis nearly impossible. It will be challenging to identify trends or common themes without significant manual coding and interpretation, which is time-consuming and prone to subjective bias. * **Potential for irrelevant answers:** Respondents might focus on aspects unrelated to the marketplace itself (e.g., ""I like online shopping in general""). This dilutes the data and makes it harder to draw meaningful conclusions about the marketplace's specific strengths. * **Bias towards positive responses:** The question only asks about what people *like* most. It doesn't allow for feedback on negative aspects or areas for improvement. A more balanced approach would include questions about both positive and negative experiences. * **Difficulty in comparing responses across different demographics:** The unstructured nature makes it hard to compare responses across different user groups (e.g., age, location, purchase frequency). To improve the question, consider using: * **Multiple choice questions:** Offer a pre-defined list of options (e.g., ""Ease of use,"" ""Product selection,"" ""Customer service,"" ""Price""). This allows for easy quantification and comparison. * **Rating scales:** Ask respondents to rate different aspects of the marketplace on a scale (e.g., 1-5 stars). * **A combination of open-ended and structured questions:** Include an open-ended question *after* the structured questions to allow for more detailed feedback on specific aspects. This approach captures both quantitative and qualitative data.","How satisfied are you with the following aspects of our online marketplace? (Please rate each on a scale of 1 to 5, where 1 is very dissatisfied and 5 i

[Learn more about working with results](https://docs.expectedparrot.com/en/latest/results.html).

## Evaluating respondents' collective answers
Next we can create a `ScenarioList` for each respondent's answers to use with our question about sentiment:

In [16]:
sl_responses = ScenarioList.from_list("responses", sl['scenarios'])
sl_responses

,responses
0,"{'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}"
1,"{'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'}"
2,"{'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.'}"


Next we add these scenarios to our sentiment question (and any others we want to add) and run it:

In [17]:
from edsl import QuestionMultipleChoice, QuestionLinearScale, Survey

q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the overall sentiment of this respondent's survey answers? {{ responses }}",
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

q_recommend = QuestionLinearScale(
    question_name = "recommend",
    question_text = "On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? {{ responses }}",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all likely", 5:"Very likely"}
)

survey = Survey([q_sentiment, q_recommend])

In [18]:
results = survey.by(sl_responses).by(m).run()

Job UUID,02b147c1-0143-4ca9-a474-91e2e72a7d46
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/02b147c1-0143-4ca9-a474-91e2e72a7d46
Exceptions Report URL,None
Results UUID,822fd422-e065-48df-a245-d1e32d11c9a2
Results URL,https://www.expectedparrot.com/content/822fd422-e065-48df-a245-d1e32d11c9a2


In [19]:
results.select("responses", "sentiment", "recommend")

,scenario.responses,answer.sentiment,answer.recommend
0,"{'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}",Very satisfied,5
1,"{'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'}",Very satisfied,5
2,"{'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.'}",Somewhat satisfied,3


## Evaluating individual answers
Next we create a `ScenarioList` for each individual question and answer to use with our question about the responsiveness of each answer. We can use the `unpivot()` method to expand the scenarios by desired identifiers (e.g., respondent ID):

In [20]:
sl_qa = sl.unpivot(id_vars = ["Respondent ID"])
sl_qa

,Respondent ID,variable,value
0,101,What do you like most about using our online marketplace?,The wide variety of products and the ease of use.
1,101,What is one feature you would like to see added to improve your shopping experience?,It would be great to have a personalized recommendation system based on my browsing history.
2,101,Can you describe a recent experience where you were dissatisfied with our service?,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it."
3,101,How do you feel about the current product search and filtering options?,The search and filtering options are intuitive and work well for me.
4,101,Is there anything else you would like to share about your experience with us?,"No, keep up the great work!"
5,102,What do you like most about using our online marketplace?,I enjoy the simplicity of the interface.
6,102,What is one feature you would like to see added to improve your shopping experience?,A feature that helps compare similar products side by side would be useful.
7,102,Can you describe a recent experience where you were dissatisfied with our service?,No complaints here.
8,102,How do you feel about the current product search and filtering options?,I find the product search to be pretty effective.
9,102,Is there anything else you would like to share about your experience with us?,I think the sky is a beautiful shade of purple today.


We can call the `rename()` method to rename the keys as desired to match our question parameters syntax:

In [21]:
sl_qa = sl_qa.rename({"Respondent ID": "id", "variable": "question", "value": "answer"})
sl_qa

,id,question,answer
0,101,What do you like most about using our online marketplace?,The wide variety of products and the ease of use.
1,101,What is one feature you would like to see added to improve your shopping experience?,It would be great to have a personalized recommendation system based on my browsing history.
2,101,Can you describe a recent experience where you were dissatisfied with our service?,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it."
3,101,How do you feel about the current product search and filtering options?,The search and filtering options are intuitive and work well for me.
4,101,Is there anything else you would like to share about your experience with us?,"No, keep up the great work!"
5,102,What do you like most about using our online marketplace?,I enjoy the simplicity of the interface.
6,102,What is one feature you would like to see added to improve your shopping experience?,A feature that helps compare similar products side by side would be useful.
7,102,Can you describe a recent experience where you were dissatisfied with our service?,No complaints here.
8,102,How do you feel about the current product search and filtering options?,I find the product search to be pretty effective.
9,102,Is there anything else you would like to share about your experience with us?,I think the sky is a beautiful shade of purple today.


In [22]:
from edsl import QuestionYesNo

q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = "Is this answer responsive to the question that was asked? Question: {{ question }} Answer: {{ answer }}"
)

In [23]:
results = q_responsive.by(sl_qa).by(m).run()

Job UUID,ac889886-fd7d-442a-8539-18745981a666
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/ac889886-fd7d-442a-8539-18745981a666
Exceptions Report URL,None
Results UUID,62702f00-9bce-43be-b85e-8f62d0110524
Results URL,https://www.expectedparrot.com/content/62702f00-9bce-43be-b85e-8f62d0110524


In [24]:
(
    results
    .filter("responsive == 'No'")
    .select("id", "question", "answer")
)

,scenario.id,scenario.question,scenario.answer
0,101,Can you describe a recent experience where you were dissatisfied with our service?,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it."
1,101,Is there anything else you would like to share about your experience with us?,"No, keep up the great work!"
2,102,Can you describe a recent experience where you were dissatisfied with our service?,No complaints here.
3,102,How do you feel about the current product search and filtering options?,I find the product search to be pretty effective.
4,102,Is there anything else you would like to share about your experience with us?,I think the sky is a beautiful shade of purple today.
5,103,How do you feel about the current product search and filtering options?,It’s okay.


## Uploading content to Coop
[Coop](https://www.expectedparrot.com/content/content/explore) is a new platform for creating, storing and sharing LLM-based research. It is fully integrated with EDSL, and a convenient place to post and access surveys, agents, results and notebooks. [Learn more about using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the contents of this notebook:

In [25]:
from edsl import Notebook

In [26]:
n = Notebook(path = "scenariolist_unpivot.ipynb")

In [27]:
info = n.push(description = "ScenarioList methods for sense checking survey data", visibility = "public")
info

{'description': 'ScenarioList methods for sense checking survey data',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/1cfaa25a-6559-4e5f-8b71-96dd4662545d',
 'uuid': '1cfaa25a-6559-4e5f-8b71-96dd4662545d',
 'version': '0.1.45.dev1',
 'visibility': 'public'}

To update an object at Coop:

In [28]:
n = Notebook(path = "scenariolist_unpivot.ipynb") # resave

In [29]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}